In [1]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

import os
import shutil
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

from keras.models import load_model

import cv2
import numpy as np

In [2]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})

var chunks = [];

function startRecording(time) {
  return new Promise(async resolve => {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    const recorder = new MediaRecorder(stream);

    recorder.ondataavailable = e => chunks.push(e.data);

    recorder.start();
    await sleep(time);

    recorder.onstop = async () => {
      const blob = new Blob(chunks);
      const text = await b2text(blob);
      resolve(text);
    };

    recorder.stop();
  });
}
"""

def record(sec=10):
    # Execute the JavaScript code for recording
    s = output.eval_js(RECORD)

    # Execute the JavaScript function to start recording
    s = output.eval_js(f'startRecording({sec*1000})')

    # Decode the base64 audio data
    b = b64decode(s.split(',')[1])

    # Specify the file path
    file_path = '/content/drive/MyDrive/Project/Testing/audio3.wav'

    # Open the file in binary write mode, create the directory if it doesn't exist
    with open(file_path, 'wb') as f:
        f.write(b)

    return file_path

record()


'/content/drive/MyDrive/Project/Testing/audio3.wav'

In [3]:
# Code for Converting my recording into Mel-Spectrogram

n_fft = 2048
hop_length = 512
n_mels = 128

y, sr = librosa.load("/content/drive/MyDrive/Project/Testing/audio3.wav")
emotion, _ = librosa.effects.trim(y)
S = librosa.feature.melspectrogram(y=emotion, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
S_DB = librosa.power_to_db(S, ref=np.max)
librosa.display.specshow(S_DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel');
plt.colorbar(format='%+2.0f dB');
plt.savefig("/content/drive/MyDrive/Project/Testing/" + 'melspec3.png')
plt.clf()

<ipython-input-3-5b5e4a8a86ee>:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load("/content/drive/MyDrive/Project/Testing/audio3.wav")
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


<Figure size 640x480 with 0 Axes>

In [4]:
# Loading the Model
from google.colab import drive
drive.mount('/content/drive')
from keras.models import load_model
import os

model_path = '/content/drive/MyDrive/Project/Speech_Emotion.h5'
if not os.path.exists(model_path):
    print(f"Error: The model file '{model_path}' does not exist.")
else:
    classifier = load_model(model_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Reading the input

input_im = cv2.imread("/content/drive/MyDrive/Project/Testing/melspec3.png")

input_im = input_im[:253 , 55: 330]
input_im = cv2.resize(input_im, (150, 150), interpolation = cv2.INTER_LINEAR)

input_im = input_im.reshape(1,150,150,3)

res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)

In [6]:
# Checking res value against different emotions

if res[0] == 0 :
    print("Angry")
elif res[0] == 1 :
    print("Disgust")
elif res[0] == 2 :
    print("Fear")
elif res[0] == 3 :
    print("Happy")
elif res[0] == 4 :
    print("Neutral")
elif res[0] == 5 :
    print("Sad")
elif res[0] == 6 :
    print("Surprise")

Fear
